In [1]:
using Printf, CSV, DataFrames, Statistics, Glob, XLSX, Dates, Plots, TimeSeries, JSON

In [2]:
directory = joinpath(homedir(), "research", "GADS_Processed", "raw_data")
f = "gen_10272_type_100_dtgrp_4_rating_1_state_Texas_raw.csv"
path = joinpath(directory, f)
df = DataFrame(CSV.File(path))

Row,EventEndDT,EventStartDT,EventTypeCode,OutageContinuationFlag,UnitID,UnitTypeCode,CommercialDT_grp,RatingMW_grp,StateName
,DateTime,DateTime,String3,Int64,Int64,Int64,Int64,Int64,String7
1,2013-06-13T02:35:00,2013-04-05T22:18:00,PO,0,10272,100,4,1,Texas
2,2013-06-14T05:50:00,2013-06-13T18:29:00,U1,0,10272,100,4,1,Texas
3,2013-07-13T09:08:00,2013-07-13T07:28:00,U1,0,10272,100,4,1,Texas
4,2013-08-07T02:18:00,2013-08-06T06:50:00,U1,0,10272,100,4,1,Texas
5,2013-10-06T03:58:00,2013-10-02T01:53:00,U2,0,10272,100,4,1,Texas
6,2014-06-03T17:30:00,2014-05-21T03:00:00,PO,0,10272,100,4,1,Texas
7,2014-07-14T04:04:00,2014-07-13T17:43:00,PO,0,10272,100,4,1,Texas
8,2015-02-17T21:30:00,2015-01-31T01:00:00,PO,0,10272,100,4,1,Texas
9,2015-03-07T13:30:00,2015-03-07T12:00:00,U1,0,10272,100,4,1,Texas


In [3]:
weather_pt = joinpath(homedir(), "research", "NOAA_WEATHER_DATA", "processed")
weather_files = glob("*.csv", weather_pt)

my_w_list = []

for w in weather_files

    # types_dict=Dict(("Column1" => DateTime))
    weather = DataFrame(CSV.File(w)) #, dateformat="yyyy-mm-dd HH:00:00"))
    weather.Column1 = DateTime.(weather.Column1, "yyyy-mm-dd HH:00:00")
    push!(my_w_list, weather)
end

weather_merged = reduce((x, y) -> innerjoin(x, y, on=:Column1), my_w_list)
DataFrames.rename!(weather_merged, :Column1 => :x)
bool_vec = weather_merged.Tmax .== 0
bool_vec = ifelse.(ismissing.(bool_vec), 0, bool_vec)
bool_vec = convert.(Bool, bool_vec)
idx1 = filter(r -> bool_vec[r], range(1, length(bool_vec)))
weather_merged[idx1, :Tmax] .= missing;


In [4]:
fname = "ercot_hourly_load_2013_2023.csv"
path = joinpath(homedir(), "research", "ERCOT_LOAD_DATA", "processed", fname)
load_data = DataFrame(CSV.File(path))
DataFrames.rename!(load_data, :Hour_End => "x")

load_data = load_data[!, [:x, :ERCOT]]

Row,x,ERCOT
,DateTime,Float64
1,2013-01-01T01:00:00,28684.9
2,2013-01-01T02:00:00,28002.7
3,2013-01-01T03:00:00,27363.8
4,2013-01-01T04:00:00,27032.4
5,2013-01-01T05:00:00,27163.8
6,2013-01-01T06:00:00,27836.8
7,2013-01-01T07:00:00,28722.0
8,2013-01-01T08:00:00,29391.7
9,2013-01-01T09:00:00,30293.8


In [5]:
x = Dict()
i = 1
mydict = Dict([("A", 1), ("RS", 1), ("NC", 1),  ("IR", 1), ("MB", 1), ("F", 2), ("U1", 2), ("U2" , 2), ("U3" , 2), ("D1", 2), ("D2", 2), ("D3", 2), ("SF", 2), ("PO", 3), ("PE", 3), ("PD", 3), ("DP", 3), ("MO", 4), ("ME", 4), ("DM", 4), ("D4", 4), ("RU", 5), ("DR", 5), ("TC", 5)])
agedict = Dict([(1, DateTime(1980, 1, 1)), (2, DateTime(1985, 1, 1)), (3, DateTime(1995, 1, 1)), (4, DateTime(2005, 1, 1)), (5, DateTime(2015, 1, 1)), (6, DateTime(2020, 1, 1))])
# agedict = Dict([(2, DateTime(1989, 1, 1)), (3, DateTime(1993, 1, 1))])
types_dict = Dict("EventStartDT" => DateTime, "EventEndDT" => DateTime)
rating_dict = Dict([(1, 50), (2, 150), (3, 200)]);
# weather_zone_dict = Dict([(2, "COAST"), (3, "NORTH_C")]);

Dict{Int64, Int64} with 3 entries:
  2 => 150
  3 => 200
  1 => 50

In [25]:
start_date_g = DateTime(2013,1,1,6)
end_date_g = DateTime(2024,1,1,6)
date_range_g = [k for k in start_date_g:Hour(1):end_date_g]

myrange_g = [k for k in 0:1:round(end_date_g - start_date_g, Hour).value]

indicator = 1

mymatch = match(r"gen_(\d+)_type_(\d+)_dtgrp_(\d+)_rating_(\d+)_state_Texas_raw.csv", f)
id = parse(Int32, mymatch[1])
type = parse(Int32, mymatch[2])
dtgrp = parse(Int32, mymatch[3])
rating = parse(Int32, mymatch[4])

# df = DataFrame(CSV.File(f, types=types_dict))
df = df[typeof.(df.EventEndDT) .== DateTime, :] # for missing DateTime values
df = df[df.EventTypeCode .!= "RS", :]

# if size(df)[1] > 1

sort!(df, :EventStartDT)
transform!(df, :EventTypeCode => ByRow(x -> mydict[x]), :EventTypeCode)

birthdate = agedict[df[1, :CommercialDT_grp]]

# 
state = convert(Array{Union{Missing, Float64}}, fill(1, length(myrange_g))) # to allow missing values
cause = convert(Array{Union{Missing, Float64}}, fill(1, length(myrange_g)))
age = [k.value for k in round.(date_range_g - birthdate, Day(1))]


# Hour that outage starts
time_vector = map(x -> round(x - start_date_g, Hour), df[!, :EventStartDT])

# Set state to zero during an event
for k in [c for c in range(1, step=1, length=length(time_vector))] # k is an index
    start_time_string = round(df[k, :EventStartDT], Hour)
    end_time_string = round(df[k, :EventEndDT], Hour)

    mycause = df[k, :EventTypeCode_function]

    # if cause == 1 # RS
    #     continue
    # end

    start_hour = time_vector[k].value # integer
    duration = round(end_time_string - start_time_string, Hour).value
    end_hour = start_hour + duration

    start_index = max(1, start_hour)

    matching_idx = findall(x -> x == end_hour, myrange_g)
    
    if length(matching_idx) < 1
        end_index = length(state)
    else
        end_index = matching_idx[1]
    end

    # If event is unit getting retired, set state as missing until the end of the period and skip to the next unit
    if mycause == 5
        state[start_index:end] .= missing
        cause[start_index:end] .= missing
        indicator = 0
    else
        state[start_index:end_index] .= 0
        cause[start_index:end_index] .= mycause
    end

end

trajectory = DataFrame(x=date_range_g, y1=state, y2=cause, y3=age)
mask = map(x -> ismissing(x) ? false : true, trajectory.y2)
trajectory = trajectory[mask, :]

merged = innerjoin(trajectory, load_data[!,[:x, :ERCOT]], on=:x)
sort!(merged, :x)
merged[!, :y4] = Dates.dayofyear.(merged[!, :x])

# Add weather, deal with Tmax = 0
merged = innerjoin(merged, weather_merged, on=:x)

# Days since last planned outage
my_data = Vector{Union{Int,Missing}}(missing, length(merged[!, :x]))

indexed = merged[merged.y2 .== 3, :]
bool_vec = diff(indexed[!, :x]) .!= Millisecond(3600000)

idx1 = filter(r -> bool_vec[r], range(1, length(bool_vec)))
event_end_datetimes = indexed[idx1, :x]

k = 1
for i in merged[!, :x]

    events_before_i = [k for k in event_end_datetimes if k < i]

    if length(events_before_i) < 1
        my_data[1:k] .= missing
    else


        val, idx2 = findmin(i .- events_before_i)
        closest_event = event_end_datetimes[idx2]
        hours_since_event = round(i - closest_event, Day).value

        my_data[k] = hours_since_event
    end

    k+= 1
end

merged[!, :y6] = my_data

# Days since last maintenance outage
my_data = Vector{Union{Int,Missing}}(missing, length(merged[!, :x]))

indexed = merged[merged.y2 .== 4, :]
bool_vec = diff(indexed[!, :x]) .!= Millisecond(3600000)

idx1 = filter(r -> bool_vec[r], range(1, length(bool_vec)))
event_end_datetimes = indexed[idx1, :x]

k = 1
for i in merged[!, :x]

    events_before_i = [k for k in event_end_datetimes if k < i]

    if length(events_before_i) < 1
        my_data[1:k] .= missing
    else
        val, idx2 = findmin(i .- events_before_i)
        closest_event = event_end_datetimes[idx2]
        hours_since_event = round(i - closest_event, Day).value

        my_data[k] = hours_since_event
    end

    k+= 1
end

merged[!, :y7] = my_data

merged[!, :y8] = map(f -> Month(f).value, merged[!, :x])
# end

81045-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 3
 3
 3
 3
 3
 4
 4
 4
 4

In [26]:
merged

Row,x,y1,y2,y3,ERCOT,y4,Pcp,Tmax,Tmin,y6,y7,y8
,DateTime,Float64?,Float64?,Int64,Float64,Int64,Float64?,Float64?,Float64?,Int64?,Int64?,Int64
1,2013-01-01T06:00:00,1.0,1.0,2922,27836.8,1,2.46,59.4,36.4,missing,missing,1
2,2013-01-01T07:00:00,1.0,1.0,2922,28722.0,1,2.46,59.4,36.4,missing,missing,1
3,2013-01-01T08:00:00,1.0,1.0,2922,29391.7,1,2.46,59.4,36.4,missing,missing,1
4,2013-01-01T09:00:00,1.0,1.0,2922,30293.8,1,2.46,59.4,36.4,missing,missing,1
5,2013-01-01T10:00:00,1.0,1.0,2922,31882.3,1,2.46,59.4,36.4,missing,missing,1
6,2013-01-01T11:00:00,1.0,1.0,2922,33359.5,1,2.46,59.4,36.4,missing,missing,1
7,2013-01-01T12:00:00,1.0,1.0,2923,34186.8,1,2.46,59.4,36.4,missing,missing,1
8,2013-01-01T13:00:00,1.0,1.0,2923,34155.1,1,2.46,59.4,36.4,missing,missing,1
9,2013-01-01T14:00:00,1.0,1.0,2923,33781.1,1,2.46,59.4,36.4,missing,missing,1


In [23]:
mask = map(x -> ismissing(x) ? false : true, merged.y2)
merged[mask, :]

Row,x,y1,y2,y3,ERCOT,y4,Pcp,Tmax,Tmin
,DateTime,Float64?,Float64?,Int64,Float64,Int64,Float64?,Float64?,Float64?
1,2013-01-01T06:00:00,1.0,1.0,2922,27836.8,1,2.46,59.4,36.4
2,2013-01-01T07:00:00,1.0,1.0,2922,28722.0,1,2.46,59.4,36.4
3,2013-01-01T08:00:00,1.0,1.0,2922,29391.7,1,2.46,59.4,36.4
4,2013-01-01T09:00:00,1.0,1.0,2922,30293.8,1,2.46,59.4,36.4
5,2013-01-01T10:00:00,1.0,1.0,2922,31882.3,1,2.46,59.4,36.4
6,2013-01-01T11:00:00,1.0,1.0,2922,33359.5,1,2.46,59.4,36.4
7,2013-01-01T12:00:00,1.0,1.0,2923,34186.8,1,2.46,59.4,36.4
8,2013-01-01T13:00:00,1.0,1.0,2923,34155.1,1,2.46,59.4,36.4
9,2013-01-01T14:00:00,1.0,1.0,2923,33781.1,1,2.46,59.4,36.4


In [20]:
mask

96403-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [22]:
length(merged.y2)

96403

In [14]:
collect(skipmissing(merged.y2 .== 3))

81045-element Vector{Bool}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [7]:
df

Row,EventEndDT,EventStartDT,EventTypeCode,OutageContinuationFlag,UnitID,UnitTypeCode,CommercialDT_grp,RatingMW_grp,StateName
,DateTime,DateTime,String3,Int64,Int64,Int64,Int64,Int64,String7
1,2013-06-13T02:35:00,2013-04-05T22:18:00,PO,0,10272,100,4,1,Texas
2,2013-06-14T05:50:00,2013-06-13T18:29:00,U1,0,10272,100,4,1,Texas
3,2013-07-13T09:08:00,2013-07-13T07:28:00,U1,0,10272,100,4,1,Texas
4,2013-08-07T02:18:00,2013-08-06T06:50:00,U1,0,10272,100,4,1,Texas
5,2013-10-06T03:58:00,2013-10-02T01:53:00,U2,0,10272,100,4,1,Texas
6,2014-06-03T17:30:00,2014-05-21T03:00:00,PO,0,10272,100,4,1,Texas
7,2014-07-14T04:04:00,2014-07-13T17:43:00,PO,0,10272,100,4,1,Texas
8,2015-02-17T21:30:00,2015-01-31T01:00:00,PO,0,10272,100,4,1,Texas
9,2015-03-07T13:30:00,2015-03-07T12:00:00,U1,0,10272,100,4,1,Texas
